In [ ]:
a = []
for i in range(1208499):
    if i % 30 == 0:
        diff = df[2].iloc[i + 1] - df[2].iloc[i]
        a.append(diff)

In [ ]:
result = a[0]
for i in range(1,len(a)):
    result += a[i]

In [ ]:
result / len(a)

# 데이터 분리하기
1. 데이터가 너무 많다는 점
2. 어느 곳에 방문할지에 대한 정보가 좀 더 중요하다는 점
3. 사람이 걸을 수 있는 정도가 한정되어 있다는 점 <br>
세가지를 고려할 때 모두를 학습시키기 보다는 일부분을 쪼개서 학습시키는 것이 합리적

In [ ]:
df_diff_30 = []
for i in range(0,len(df),30):
    df_diff_30.append(df.iloc[i])

In [ ]:
df_diff_30_pd = pd.DataFrame(df_diff_30)

In [ ]:
x, y = df_128_30[0], df_128_30[1]

In [ ]:
import random
import numpy as np
import torch

# multivariate data preparation
from numpy import array
from numpy import hstack
 
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)-n_steps):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-2], sequences[end_ix, -2:]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)
 
# x, y
# define input sequence
in_seq1 = x.values
in_seq2 = y.values

# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))

# horizontally stack columns
dataset = hstack((in_seq1, in_seq2, in_seq1, in_seq2))

In [ ]:
df = split_sequences(dataset, 5)

# lstm 모델 학습
    # 어려웠던 점
    1. 인풋 사이즈를 맞춰줘야 하는데 코드를 온전히 이해하여야 했기에 많은 어려움이 존재
    2. 시계열 데이터이기 때문에 마지막 배치의 경우에 문제가 존재했음 -> 이 부분을 처리하는 것이 중요!
    3. 텐서 자체에서 문제가 존재했고, 데이터를 텐서로 변환하는 과정에서 소수점의 사라짐이 존재하게 되었음 이러한 문제점을 해결하는 방법은? torch.set_printoptions(precision=10)으로 소수점을 사라지지 않게 하는 방법
    4. 1회 학습시 gpu를 사용해야 했는데, 모델, 파라미터, loss function에 모두 cuda를 할당해줘야 하지만 처음에는 파라미터에만 할당해줘서 어려움이 있었음.

In [ ]:
cuda0 = torch.device('cuda:0')

In [ ]:
torch.set_printoptions(precision=10)

In [ ]:
class MV_LSTM(torch.nn.Module):
    def __init__(self,n_features,seq_length):
        super(MV_LSTM, self).__init__()
        self.n_features = n_features
        self.seq_len = seq_length
        self.n_hidden = 100 # number of hidden states
        self.n_layers = 300 # number of LSTM layers (stacked)
    
        self.l_lstm = torch.nn.LSTM(input_size = n_features, 
                                 hidden_size = self.n_hidden,
                                 num_layers = self.n_layers, 
                                batch_first = True,dropout=0.1).to(device=cuda0)
        # according to pytorch docs LSTM output is 
        # (batch_size,seq_len, num_directions * hidden_size)
        # when considering batch_first = True
        self.l_linear = torch.nn.Linear(self.n_hidden*self.seq_len, 1).to(device=cuda0)
        
    
    def init_hidden(self, batch_size):
        # even with batch_first = True this remains same as docs
        hidden_state = torch.zeros(self.n_layers,batch_size, self.n_hidden, device = cuda0)
        cell_state = torch.zeros(self.n_layers,batch_size, self.n_hidden, device = cuda0)
        self.hidden = (hidden_state, cell_state)
    
    
    def forward(self, x):
        batch_size, seq_len, _ = x.size()
        
        lstm_out, self.hidden = self.l_lstm(x, self.hidden)
        # lstm_out(with batch_first = True) is 
        # (batch_size,seq_len,num_directions * hidden_size)
        # for following linear layer we want to keep batch_size dimension and merge rest       
        # .contiguous() -> solves tensor compatibility error
        x = lstm_out.contiguous().view(batch_size,-1)
        return self.l_linear(x)

In [ ]:
# 모델 구성 방법
    데이터들을 

In [ ]:
n_features = 1 # this is number of parallel inputs
n_timesteps = 5 # this is number of timesteps

# convert dataset into input/output
X, y = split_sequences(dataset, n_timesteps)
print(X.shape, y.shape)

# create NN
mv_net_x = MV_LSTM(n_features, n_timesteps)
mv_net_y = MV_LSTM(n_features, n_timesteps)
criterion = torch.nn.MSELoss().to(device=cuda0)
 # reduction='sum' created huge loss value
optimizer_x = torch.optim.Adam(mv_net_x.parameters(), lr=1e-1)
optimizer_y = torch.optim.Adam(mv_net_y.parameters(), lr=1e-1)

train_episodes = 30
batch_size = 50

In [ ]:
mv_net_x.train()
mv_net_y.train()

for t in range(train_episodes):
    for b in range(0,len(X),batch_size):
        inpt = X[b:b+batch_size,:,:]
        target = y[b:b+batch_size]

        # x, y 트레인셋
        x_batch = torch.tensor(inpt[:,:,0],dtype=torch.float32, device = cuda0) # 텐서로 바꾸면서
        y_batch = torch.tensor(inpt[:,:,1],dtype=torch.float32, device = cuda0)
        
        if x_batch.shape == (batch_size,n_timesteps):
            x_batch = x_batch.reshape(batch_size,n_timesteps,1) # 3 채널로 맞춰줘야 함
            y_batch = y_batch.reshape(batch_size,n_timesteps,1)

            mv_net_x.init_hidden(x_batch.size(0))
            mv_net_y.init_hidden(y_batch.size(0))

            # x,y의 예측 값
            output_x = mv_net_x(x_batch)
            output_y = mv_net_y(y_batch)

            # x,y의 정답
            target_x_batch = torch.tensor(target[:, 0],dtype=torch.float32, device = cuda0)
            target_y_batch = torch.tensor(target[:, 1],dtype=torch.float32, device = cuda0)


        #    lstm_out, _ = mv_net.l_lstm(x_batch,nnet.hidden)    
        #    lstm_out.contiguous().view(x_batch.size(0),-1)

            # loss function이 두개의 합이 최소가 되는 쪽으로 loss를 구함
            loss = criterion(output_x.view(-1), target_x_batch) + criterion(output_y.view(-1), target_y_batch)
            loss.backward()

            optimizer_x.step()        
            optimizer_x.zero_grad() # x가 0으로 흐르도록 함

            optimizer_y.step()
            optimizer_y.zero_grad() # y가 0으로 흐르도록 함
    print('step : ' , t , 'loss : ' , loss.item())